In [2]:
# import TF and TF Hub libraries.
import tensorflow as tf
import tensorflow_hub as hub

# load the input image.
image_path = './image.jpg'

# reads the image from the specified path as a raw byte string
image = tf.io.read_file(image_path)

# converts the raw byte string into a tensor representing the image, making it usable for further processing
image = tf.compat.v1.image.decode_jpeg(image)

# this line adds a new dimension at the start of the tensor shape, making it [1, height, width, channels]
image = tf.expand_dims(image, axis=0)

# resize and pad the image to keep the aspect ratio and fit the expected size.
image = tf.cast(tf.image.resize_with_pad(image, 256, 256), dtype=tf.int32)

# Download the model from TF Hub.
model = hub.load("https://www.kaggle.com/models/google/movenet/TensorFlow2/multipose-lightning/1")

# this retrieves the specific callable signature for running inference on the model
movenet = model.signatures['serving_default']

# Run model inference.
outputs = movenet(image)

# Output is a [1, 6, 56] tensor => retrieves the predicted keypoints, which are structured in a tensor with dimensions indicating the number of people and their respective keypoints
keypoints = outputs['output_0']

2024-07-12 22:31:01.118629: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 22:31:01.119400: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
# Load the TensorFlow SavedModel
saved_model_dir = "./movenet-tensorflow2-multipose-lightning-v1/"  # Directory containing saved_model.pb

# Create the TFLiteConverter object
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# Enable TensorFlow ops that are not natively supported by TensorFlow Lite
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # TensorFlow Lite ops
    tf.lite.OpsSet.SELECT_TF_OPS  # Enable TensorFlow ops
]

# Optional: Optimization for size or latency
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model
tflite_model = converter.convert()

# Save the TFLite model
tflite_model_path = "model.tflite"
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print("TFLite model conversion complete and saved as model.tflite")

TFLite model conversion complete and saved as model.tflite


W0000 00:00:1720812954.404284   28722 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1720812954.404299   28722 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-07-12 22:35:54.404429: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ./movenet-tensorflow2-multipose-lightning-v1/
2024-07-12 22:35:54.464816: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-12 22:35:54.464858: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: ./movenet-tensorflow2-multipose-lightning-v1/
2024-07-12 22:35:54.787205: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2024-07-12 22:35:55.144885: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: ./movenet-tensorflow2-multipose-lightning-v1/
2024-07-12 22:35:55.309915: I tensorflow/cc/saved_model/loader.cc:317] SavedModel load for tags { serve }; Stat

In [15]:
import cv2
import numpy as np 

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print input details to verify the expected input shape
print(f"Input details: {input_details}")
input_shape = input_details[0]['shape']  # Get the input shape
print(f"Expected input shape: {input_shape}")

# Load and preprocess the image
image_path = './image.jpg'
print(f"Loading image from: {image_path}")
image = cv2.imread(image_path)

# Check if the image was loaded properly
if image is None:
    raise ValueError(f"Could not read the image from path: {image_path}")

print(f"Original image shape: {image.shape}")

# Correctly resize the image
input_height, input_width = input_shape[1:3]
print(f"Resizing image to: {input_width}x{input_height}")
input_data = cv2.resize(image, (input_width, input_height))

# Convert image to the expected data type and normalize if required
input_data = np.expand_dims(input_data, axis=0).astype(input_details[0]['dtype'])
print(f"Input data shape after resizing: {input_data.shape}")

# Normalization (if required by your model, you might need to adjust this)
# input_data = input_data / 255.0

# Set the tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get the results
output_data = interpreter.get_tensor(output_details[0]['index'])
print("Inference output:")
print(output_data) # X Y confidence

Input details: [{'name': 'serving_default_input:0', 'index': 0, 'shape': array([1, 1, 1, 3], dtype=int32), 'shape_signature': array([ 1, -1, -1,  3], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Expected input shape: [1 1 1 3]
Loading image from: ./image.jpg
Original image shape: (1195, 1200, 3)
Resizing image to: 1x1
Input data shape after resizing: (1, 1, 1, 3)
Inference output:
[[[ 1.19312555e-01  3.21222320e-02  4.10582311e-03  9.23214406e-02
    1.52409583e-01  1.84597715e-03 -5.12045510e-02  1.06847323e-01
    3.64131120e-05  1.95450976e-01  4.48140353e-01  5.31197912e-08
   -7.73408860e-02  3.29142034e-01  2.07998901e-14  3.80024672e-01
    3.01253885e-01  9.79928378e-13 -5.20988554e-02  2.61832356e-01
    4.78906031e-07 -4.07879561e-01  3.00798893e-01  1.68324711e-37
   -6.32079720e-01  2.4495428

2024-07-12 22:40:40.964105: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 22:40:40.964302: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-07-12 22:40:40.964977: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_inter_op_parallelism which is not in the op definition: Op<name=StridedSlice; signature=input:T, begin:Index, end:Index, strides:Index -> output:T; attr=T:type; a

In [16]:
# Print input tensor details
print(f"Input details: {input_details}")

Input details: [{'name': 'serving_default_input:0', 'index': 0, 'shape': array([1, 1, 1, 3], dtype=int32), 'shape_signature': array([ 1, -1, -1,  3], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [17]:
# Assuming the output is a 1D array of logits
logits = output_data[0]

# Apply softmax to get probabilities
probabilities = tf.nn.softmax(logits).numpy()

# Get the predicted class
predicted_class = np.argmax(probabilities) # identifies the index of the maximum value in the probabilities array.
print(f"Predicted class: {predicted_class}")
print(f"Class probabilities: {probabilities}")

Predicted class: 109
Class probabilities: [[0.01781955 0.01633166 0.01588046 0.01734501 0.01841919 0.01584461
  0.01502595 0.0175988  0.01581596 0.01922929 0.02475739 0.01581539
  0.01463832 0.02197983 0.01581539 0.02312717 0.02137533 0.01581539
  0.01501252 0.02054907 0.0158154  0.01051817 0.02136561 0.01581539
  0.00840565 0.02020515 0.01581539 0.01392872 0.02065367 0.01581539
  0.01208684 0.02127294 0.01581539 0.02375426 0.01626263 0.01581647
  0.01565693 0.01944094 0.01581761 0.01009122 0.01514702 0.01581539
  0.00999383 0.01681065 0.01581539 0.01427472 0.01943724 0.01581539
  0.01457523 0.01930341 0.0158154  0.01581539 0.04146511 0.0300853
  0.04146511 0.01581539]
 [0.01745252 0.01599528 0.01555337 0.01698776 0.01803982 0.01551826
  0.01471647 0.01723633 0.01549021 0.01883323 0.02424747 0.01548965
  0.01433682 0.02152712 0.01548965 0.02265083 0.02093507 0.01548965
  0.01470331 0.02012583 0.01548965 0.01030153 0.02092554 0.01548965
  0.00823252 0.019789   0.01548965 0.01364184 0.02